Importing required library

In [2]:
import numpy as np 
from zipfile import ZipFile
import keras.utils as ku
from sklearn.model_selection import train_test_split
from tensorflow.keras.utils import to_categorical 
from keras.models import Sequential #Initialise our neural network model as a sequential networkfrom
from keras.layers import Conv2D #Convolution operation
from keras.layers.normalization import BatchNormalization
from keras.regularizers import l2
from keras.layers import Activation#Applies activation function
from keras.layers import Dropout#Prevents overfitting by randomly converting few outputs to zero
from keras.layers import MaxPooling2D # Maxpooling function
from keras.layers import Flatten # Converting 2D arrays into a 1D linear vector
from keras.layers import Dense # Regular fully connected neural network
from keras import optimizers
from keras.callbacks import ReduceLROnPlateau, EarlyStopping, TensorBoard, ModelCheckpoint
from sklearn.metrics import accuracy_score
from keras.regularizers import l2

Defining and initializing variables

In [3]:

DATA_PATH = 'fer2013.csv'
TRAIN_DIR = 'data/train'
VALID_DIR = 'data/valid'
TEST_DIR = 'data/test'
MODEL_DIR = '/model'
shape_predictor_68_face_landmarks = 'shape_predictor_68_face_landmarks.dat'

img_size = 48
crop_size = 44

class_names = ['Angry', 'Disgust', 'Fear', 'Happy', 'Sad', 'Surprise', 'Neutral']

num_classes = len(class_names)

Loading image DataBase file from specified location

In [21]:
def load_data(dataset_path):
  data = []  
  test_data = []  
  test_labels = []  
  labels =[]
  #filename = "fer2013.csv.zip"
  #with ZipFile(filename, 'r') as zip: 
  #  zip.extractall()
  with open(dataset_path, 'r') as file:      
    for line_no, line in enumerate(file.readlines()):          
      if 0 < line_no <= 35887:            
        curr_class, line, set_type = line.split(',')            
        image_data = np.asarray([float(x) for x in line.split()]).reshape(48, 48)            
        image_data =image_data.astype(np.uint8)/255.0                        
        if (set_type.strip() == 'PrivateTest'):
          test_data.append(image_data)              
          test_labels.append(curr_class)            
        else:              
          data.append(image_data)              
          labels.append(curr_class)            
    test_data = np.expand_dims(test_data, -1)      
    test_labels = to_categorical(test_labels, num_classes = 7)      
    data = np.expand_dims(data, -1)         
    labels = to_categorical(labels, num_classes = 7)          
  return np.array(data), np.array(labels), np.array(test_data), np.array(test_labels)

    #print("Number of images in Training set:", len(train_data))print("Number of images in Test set:", len(test_data))

Unzipping the image database on root directory

In [5]:
#def load_data():
#  filename = "fer2013.csv.zip"
#  with ZipFile(filename, 'r') as zip:
 #   zip.extractall()
#load_data()

Helper functions

In [12]:
lr_reducer = ReduceLROnPlateau(monitor='val_loss', factor=0.9, patience=3)
early_stopper = EarlyStopping(monitor='val_acc', min_delta=0, patience=6, mode='auto')
checkpointer = ModelCheckpoint('Model/weights.hd5', monitor='val_loss', verbose=1, save_best_only=True)

Building ML model fitting/compiling

In [ ]:
epochs = 10
batch_size = 64
learn_rate = 0.001
model = Sequential()
#Layer 1
model.add(Conv2D(64, (5, 5), activation='relu', input_shape=(48, 48, 1), kernel_regularizer=l2(0.01)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

#Layer 2
model.add(Conv2D(128, (5, 5), padding='same',activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

#Layer 3
model.add(Conv2D(256, (3, 3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2,2), strides=(2, 2)))

#Layer 4
model.add(Dense(128, activation='relu'))
model.add(Flatten())
model.add(BatchNormalization())
model.add(Dropout(0.25))
model.add(Dense(7, activation='softmax'))

adam = optimizers.Adam(learning_rate = learn_rate)
#compiling the model
model.compile(optimizer = adam, loss = 'categorical_crossentropy', metrics = ['accuracy'])    
train_data, train_labels, test_data, test_labels = load_data('fer2013.csv')
model.fit(train_data, train_labels, epochs = epochs, batch_size = batch_size, 
          validation_split = 0.1, 
          shuffle = True, 
          callbacks=[lr_reducer, checkpointer, early_stopper]
          )
print("Number of images in Training set:", len(train_data))
print("Number of images in Test set:", len(test_data))
predicted_test_labels = np.argmax(model.predict(test_data)) np.argmax(model.predict(test_data), axis=1)


Making predictions from built model

In [29]:
predicted_test_labels = np.argmax(model.predict(test_data), axis=1)
test_labels = np.argmax(test_labels, axis=1)
print ("Accuracy score = ", accuracy_score(test_labels, predicted_test_labels))


Accuracy score =  0.5444413485650599
